In [2]:
import sys
import os

mitie_path = os.environ['MITIE_HOME'] + "\\"
sys.path.append(mitie_path)


from mitie import *
from collections import defaultdict

D:\Program Files\mitie\


In [3]:
print "loading NER model..."
ner = named_entity_extractor(mitie_path + 'MITIE-models/english/ner_model.dat')
print "\nTags output by this NER model:", ner.get_possible_ner_tags()

# Load a text file and convert it into a list of words.  
tokens = tokenize(load_entire_file(mitie_path + 'sample_text.txt'))
print "Tokenized input:", tokens

loading NER model...

Tags output by this NER model: ['PERSON', 'LOCATION', 'ORGANIZATION', 'MISC']
Tokenized input: ['A', 'Pegasus', 'Airlines', 'plane', 'landed', 'at', 'an', 'Istanbul', 'airport', 'Friday', 'after', 'a', 'passenger', '"', 'said', 'that', 'there', 'was', 'a', 'bomb', 'on', 'board', '"', 'and', 'wanted', 'the', 'plane', 'to', 'land', 'in', 'Sochi', ',', 'Russia', ',', 'the', 'site', 'of', 'the', 'Winter', 'Olympics', ',', 'said', 'officials', 'with', 'Turkey', "'s", 'Transportation', 'Ministry', '.', 'Meredith', 'Vieira', 'will', 'become', 'the', 'first', 'woman', 'to', 'host', 'Olympics', 'primetime', 'coverage', 'on', 'her', 'own', 'when', 'she', 'fills', 'on', 'Friday', 'night', 'for', 'the', 'ailing', 'Bob', 'Costas', ',', 'who', 'is', 'battling', 'a', 'continuing', 'eye', 'infection', '.', '"', 'It', "'s", 'an', 'honor', 'to', 'fill', 'in', 'for', 'him', ',', '"', 'Vieira', 'said', 'on', 'TODAY', 'Friday', '.', '"', 'You', 'think', 'about', 'the', 'Olympics', ','

In [4]:
entities = ner.extract_entities(tokens)
print "\nEntities found:", entities
print "\nNumber of entities detected:", len(entities)


Entities found: [(xrange(1, 3), 'ORGANIZATION'), (xrange(7, 8), 'LOCATION'), (xrange(30, 31), 'LOCATION'), (xrange(32, 33), 'LOCATION'), (xrange(44, 45), 'LOCATION'), (xrange(46, 48), 'ORGANIZATION'), (xrange(49, 51), 'PERSON'), (xrange(58, 59), 'MISC'), (xrange(73, 75), 'PERSON'), (xrange(96, 97), 'PERSON'), (xrange(107, 108), 'MISC'), (xrange(116, 118), 'PERSON'), (xrange(121, 122), 'PERSON'), (xrange(139, 141), 'ORGANIZATION'), (xrange(143, 145), 'PERSON'), (xrange(150, 151), 'LOCATION'), (xrange(159, 161), 'PERSON'), (xrange(164, 166), 'PERSON'), (xrange(169, 170), 'LOCATION'), (xrange(187, 188), 'LOCATION'), (xrange(192, 194), 'PERSON'), (xrange(204, 206), 'PERSON'), (xrange(210, 211), 'LOCATION')]

Number of entities detected: 23


In [9]:
# entities is a list of tuples, each containing an xrange that indicates which
# tokens are part of the entity, the entity tag, and an associate score.  The
# entities are also listed in the order they appear in the input text file.
# Here we just print the score, tag, and text for each entity to the screen.
# The larger the score the more confident MITIE is in its prediction.
for e in entities:
    range = e[0]
    tag = e[1]
    entity_text = " ".join(tokens[i] for i in range)
    print tag + ": " + entity_text

ORGANIZATION: Pegasus Airlines
LOCATION: Istanbul
LOCATION: Sochi
LOCATION: Russia
LOCATION: Turkey
ORGANIZATION: Transportation Ministry
PERSON: Meredith Vieira
MISC: Olympics
PERSON: Bob Costas
PERSON: Vieira
MISC: Olympics
PERSON: Bob Costas
PERSON: Bob
ORGANIZATION: NBC Olympics
PERSON: Jim Bell
LOCATION: Sochi
PERSON: Josiah Franklin
PERSON: Benjamin Franklin
LOCATION: Boston
LOCATION: Philadelphia
PERSON: Benjamin Franklin
PERSON: Benjamin Franklin
LOCATION: Boston


In [10]:
# Now let's run one of MITIE's binary relation detectors.  MITIE comes with a
# bunch of different types of relation detector and includes tools allowing you
# to train new detectors.  However, here we simply use one, the "person born in
# place" relation detector.
rel_detector = binary_relation_detector(mitie_path + "MITIE-models/english/binary_relations/rel_classifier_people.person.place_of_birth.svm")

# First, let's make a list of neighboring entities.  Once we have this list we
# will ask the relation detector if any of these entity pairs is an example of
# the "person born in place" relation. 
neighboring_entities = [(entities[i][0], entities[i+1][0]) for i in xrange(len(entities)-1)]

In [11]:
# Also swap the entities and add those in as well.  We do this because "person
# born in place" mentions can appear in the text in as "place is birthplace of
# person".  So we must consider both possible orderings of the arguments.
neighboring_entities += [(r,l) for (l,r) in neighboring_entities]

# Now that we have our list, let's check each entity pair and see which one the
# detector selects.
for person, place in neighboring_entities:
    # Detection has two steps in MITIE. First, you convert a pair of entities
    # into a special representation.
    rel = ner.extract_binary_relation(tokens, person, place)
    # Then you ask the detector to classify that pair of entities.  If the
    # score value is > 0 then it is saying that it has found a relation.  The
    # larger the score the more confident it is.  Finally, the reason we do
    # detection in two parts is so you can reuse the intermediate rel in many
    # calls to different relation detectors without needing to redo the
    # processing done in extract_binary_relation().
    score = rel_detector(rel)
    # Print out any matching relations.
    if (score > 0):
        person_text     = " ".join(tokens[i] for i in person)
        birthplace_text = " ".join(tokens[i] for i in place)
        print person_text, "BORN_IN", birthplace_text

Benjamin Franklin BORN_IN Boston
Benjamin Franklin BORN_IN Boston
Benjamin Franklin BORN_IN Philadelphia


In [12]:
# The code above shows the basic details of MITIE's relation detection API.
# However, it is important to note that real world data is noisy any confusing.
# Not all detected relations will be correct.  Therefore, it's important to
# aggregate many relation detections together to get the best signal out of
# your data.  A good way to do this is to pick an entity you are in interested
# in (e.g. Benjamin Franklin) and then find all the relations that mention him
# and order them by most frequent to least frequent.  We show how to do this in
# the code below.
query = "Benjamin Franklin"
hits = defaultdict(int)

for person, place in neighboring_entities:
    rel = ner.extract_binary_relation(tokens, person, place)
    score = rel_detector(rel)
    if (score > 0):
        person_text     = " ".join(tokens[i] for i in person)
        birthplace_text = " ".join(tokens[i] for i in place)
        if (person_text == query):
            hits[birthplace_text] += 1

print "\nTop most common relations:"
for place, count in sorted(hits.iteritems(), key=lambda x:x[1], reverse=True):
    print count, "relations claiming", query, "was born in", place


Top most common relations:
2 relations claiming Benjamin Franklin was born in Boston
1 relations claiming Benjamin Franklin was born in Philadelphia
